In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd
from sklearn.preprocessing import LabelEncoder

cred = credentials.Certificate('kiosk-firestore-jnsy-bcfe4-firebase-adminsdk-269q4-4a795774bb.json')
firebase_admin.initialize_app(cred)

In [ ]:
#데이터 전처리까지..

In [2]:
# 감정 키워드는 원 핫 인코딩으로 데이터 전처리한다.
#from sklearn.preprocessing import OneHotEncoder
#일단은 첫번째감정+두번째감정 으로 저장하는데 가중치 고려할 방법 있었으면 좋겠다
#이 방법은 순서를 중요시하고, or로 합하는 방법은 단순 평균!

# 8가지 감정 키워드
emotion_keyword = ["anger","contempt","disgust","fear","happiness","neutral","sadness","surprise"]

emotion_df = pd.DataFrame({ 'emotion_keyword':emotion_keyword })
emotion_vec = pd.get_dummies(emotion_df).values
#print(emotion_vec)

emotion_keyword_to_vec = {}
for index,keyword in enumerate(emotion_keyword):
    emotion_keyword_to_vec[keyword] = ''.join(list(map(str,emotion_vec[index].tolist())))
    
print(emotion_keyword_to_vec)

{'anger': '10000000', 'contempt': '01000000', 'disgust': '00100000', 'fear': '00010000', 'happiness': '00001000', 'neutral': '00000100', 'sadness': '00000010', 'surprise': '00000001'}


In [ ]:
# 사이킷런 원 핫 인코딩 연습? (실패..)
#from sklearn.preprocessing import OneHotEncoder,LabelEncoder
#import numpy as np

#emotion_keyword = ["anger","contempt","disgust","fear","happiness","neutral","sadness","surprise"]

#encoder = LabelEncoder()
#encoder.fit(emotion_keyword)
#labels = encoder.transform(emotion_keyword)
#labels = labels.reshape(-1,1)

#oh_encoder = OneHotEncoder()
#oh_encoder.fit(labels)
#oh_labels = oh_encoder.transform(labels)

#print(oh_labels.toarray())
#print(type(oh_labels.toarray()))


In [ ]:
#print(oh_labels.toarray()[0]|oh_labels.toarray()[1])

In [3]:
docs = firestore.client().collection(u'order').stream() #get 사용하지 말란다

doc_list = []
for doc in docs:
    #print(doc.get('emotion'))
    #print(type(doc))
    doc_list_row = []
    
    # 메뉴 - 레이블
    doc_list_row.append(doc.get('orderToString'))
 
    # 날씨
    weather_key_list = []
    for key in doc.get('weather').keys():
        #weather_key_list.append(doc.get('weather').get(key))
        doc_list_row.append(doc.get('weather').get(key))
    
    # 감정
    emotion_key_list = ''
    for key in doc.get('emotion').keys():
        #print(key)
        emotion_key_list += emotion_keyword_to_vec.get(key)
    doc_list_row.append(emotion_key_list)
    
    doc_list.append(doc_list_row)
#print(doc_list)
    

In [40]:
doc_df = pd.DataFrame(doc_list, columns=['item_name','w_humidity','w_temp','w_speed','emotion'])
doc_df

,item_name,w_humidity,w_temp,w_speed,emotion
0,삼계탕/휘핑:0.0/샷:0.5/,79.0,304.51,1.50,0100000000000100
1,삼계탕/휘핑:0.0/샷:0.5/,79.0,304.51,1.50,0100000000000100
2,삼계탕/휘핑:0.0/샷:0.5/,79.0,304.51,1.50,0100000000000100
3,라떼/휘핑:0.0/샷:2.0/,94.0,296.60,0.50,0000001000000100
4,삼계탕/휘핑:0.0/샷:0.5/,79.0,304.51,1.50,0100000000000100
5,삼계탕/휘핑:0.0/샷:0.5/,79.0,304.51,1.50,0100000000000100
6,라떼/휘핑:0.0/샷:0.5/,88.0,298.49,2.26,0000010000000001
7,삼계탕/휘핑:0.0/샷:0.5/,79.0,304.51,1.50,0100000000000100
8,라떼/휘핑:0.0/샷:0.5/,88.0,298.49,2.26,0000010000000001
9,라떼/휘핑:0.0/샷:0.5/,100.0,295.78,0.50,0100000000000100


In [35]:
#레이블 인코딩
encoder = LabelEncoder()
items = doc_df['item_name'].values.tolist()
encoder.fit(items)
labels = encoder.transform(items)

print('인코딩 클래스:', encoder.classes_)
print('디코딩 원본 값:', encoder.inverse_transform([0,1]))

{'삼계탕/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:2.0/'}
3


In [37]:
print(list(set(label_list)))
#print(len([set(label_list)]))

['삼계탕/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:2.0/']


In [41]:
label_list = doc_df['item_name'].values.tolist()
label_df = pd.DataFrame({ 'label_vec':label_list })

doc_df['item_name'] = pd.get_dummies(label_df) #one hot encoding으로 대체


0     0
1     0
2     0
3     0
4     0
5     0
6     1
7     0
8     1
9     1
10    1
11    0
12    0
13    0
14    1
15    1
16    0
17    0
18    0
Name: item_name, dtype: uint8

In [19]:
print(label_list)
print(set(label_list))
print(len(set(label_list)))

['삼계탕/휘핑:0.0/샷:0.5/', '삼계탕/휘핑:0.0/샷:0.5/', '삼계탕/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:2.0/', '삼계탕/휘핑:0.0/샷:0.5/', '삼계탕/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:0.5/', '삼계탕/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:0.5/', '삼계탕/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:2.0/', '삼계탕/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:0.5/', '삼계탕/휘핑:0.0/샷:0.5/', '삼계탕/휘핑:0.0/샷:0.5/', '삼계탕/휘핑:0.0/샷:0.5/']
{'삼계탕/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:0.5/', '라떼/휘핑:0.0/샷:2.0/'}
3


In [6]:
doc_df[['w_humidity','w_temp','w_speed','emotion']]

,w_humidity,w_temp,w_speed,emotion
0,79.0,304.51,1.50,0100000000000100
1,79.0,304.51,1.50,0100000000000100
2,79.0,304.51,1.50,0100000000000100
3,94.0,296.60,0.50,0000001000000100
4,79.0,304.51,1.50,0100000000000100
5,79.0,304.51,1.50,0100000000000100
6,88.0,298.49,2.26,0000010000000001
7,79.0,304.51,1.50,0100000000000100
8,88.0,298.49,2.26,0000010000000001
9,100.0,295.78,0.50,0100000000000100


In [7]:
doc_df[['item_name']]

,item_name
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,0
8,1
9,1


In [8]:
# 알고리즘: 결정 트리 (임시)
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
#import warnings
#warnings.filterwarnings('ignore')

dt_clf = DecisionTreeClassifier()

X_train,X_test,y_train,y_test = train_test_split(doc_df[['w_humidity','w_temp','w_speed','emotion']],
                                                doc_df[['item_name']], test_size=0.2)

dt_clf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [15]:
from sklearn.metrics import accuracy_score

pred = dt_clf.predict(X_test)
print(pred)
print(y_test)
accuracy = accuracy_score(y_test, pred)
print('결정 트리 예측 정확도: {0:.4f}'.format(accuracy))

[0 0 0 0]
    item_name
1           0
7           0
18          0
9           1
결정 트리 예측 정확도: 0.7500


In [10]:
print(type(X_test))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
print(type(doc_df['emotion'].iloc[0]))

In [16]:
# 람다 test
label_df = pd.DataFrame({ 'w_humidity':[80.0],
                         'w_temp':[300.0],
                         'w_speed':[1.00],
                         'emotion':['0100000000000001'] })
# home_recom/80.0/300.0/1.00/'0100000000000001'
print(label_df)
pred2 = dt_clf.predict(label_df)
print(pred2)

   w_humidity  w_temp  w_speed           emotion
0        80.0   300.0      1.0  0100000000000001
[0]


In [ ]:
#시각화 출력파일 생성
from sklearn.tree import export_graphviz

export_graphviz(dt_clf, out_file='tree.dot',feature_names=['w_humidity','w_temp','w_speed','emotion'], impurity=True,filled=True)
#'item_name','w_humidity','w_temp','w_speed','emotion'
#class_names=['item_name'],feature_names=['w_humidity','w_temp','w_speed','emotion']

In [ ]:
import graphviz

with open('tree.dot') as f:
    dot_graph = f.read()
    
graphviz.Source(dot_graph)